In [1]:
from dotenv import load_dotenv
import os
import nest_asyncio
nest_asyncio.apply()

load_dotenv()
api_key=os.getenv("MISTRAL_API_KEY")

In [2]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.llms.mistralai import MistralAI
from llama_index.core import Settings
from IPython.display import Markdown

#Setting up model
embed_model= MistralAIEmbedding(api_key=api_key)
llm= MistralAI(api_key=api_key)
Settings.llm= llm
Settings.embed_model=embed_model

In [3]:
#Reading file name
files=[file for file in os.listdir("../data/ict-policy/processed") if file.endswith('.txt')]

var_names=[]
#Checking the results
for i, file in enumerate(files):
    var_name=file.split('.')[0]
    var_names.append(var_name)
    print(var_name)

MAS
RBI
BB


In [4]:
#Reading documents
docs= {}
for doc in var_names:
    docs[doc]=SimpleDirectoryReader(input_files=["../data/ict-policy/processed/{}.txt".format(doc)]).load_data()

In [5]:
#Checking documents
for doc in var_names:
    print(doc)
    print(docs[doc][0].text[:100])

MAS
# 1 Preface

1.1 The technology landscape of the financial sector is transforming at a rapid pace an
RBI
# Chapter I - Preliminary

1 Short Title and Commencement

(a) These Directions shall be called the 
BB
# Preface

The technology landscape of the financial sector is changing at a rapid pace and the unde


In [6]:
import re

def split_by_chapters(text):
    # Step 1: Split text into chapters by `#`
    chapters = re.split(r"(# .+)", text)  # Match chapter titles starting with `# `
    chapter_chunks = []

    for i in range(1, len(chapters), 2):  # Iterate over matched chapter titles and their content
        chapter_title = chapters[i].strip()
        chapter_content = chapters[i + 1].strip()
        # chapter_chunks.append(f"{chapter_title}\n{chapter_content}")
        chapter_chunks.append({
            "title": chapter_title,
            "content": chapter_content
        })


    return chapter_chunks

In [7]:
# Split the text into chapterwise chunks
chapter_chunks= {}
for doc in var_names:
    chapter_chunks[doc] = split_by_chapters(docs[doc][0].text)

In [12]:
#Checking chunk of all docs
for doc in var_names:
    for i, chunk in enumerate(chapter_chunks[doc][:2]):
        print(f"{doc}_Chunk: {i+1}")
        print(chunk)

MAS_Chunk: 1
{'title': '# 1 Preface', 'content': '1.1 The technology landscape of the financial sector is transforming at a rapid pace and the underlying information technology (IT) infrastructure supporting financial services has grown in scope and complexity in recent years. Many financial institutions (FIs) are riding the wave of digitalisation to increase operational efficiency and to deliver better services to consumers.\n\n1.2 Digital transformation in the financial sector can be broadly characterised by the adoption of new technology and the use of existing technology in innovative ways to achieve greater automation and enrich financial service offerings.\n\n1.3 While digital transformation brings significant benefits to the financial ecosystem, it also increases FIs’ exposure to a range of technology risks, including cyber risk. The techniques used by cyber threat actors are becoming increasingly sophisticated, and weak links in the interconnected financial ecosystem can be com

In [13]:
#Importing library for splitting
from llama_index.core import Document
import nltk

def sentence_split(chapter_chunks):
    nodes= []
    for section in chapter_chunks:
        # Preprocess the text to replace multiple consecutive newline characters with a single newline character
        content = re.sub(r'\n{2,}', '\n', section["content"])
        # Preprocess the text to replace newline characters with a space
        content = re.sub(r'(?<!\.)\n', '.\n', content)
        sentences=nltk.sent_tokenize(content)
        for sentence in sentences:
            node= Document(text=sentence,
                        metadata={
                            "title": section["title"]
                        })
            nodes.append(node)
    return nodes

In [14]:
# Splitting chapter_chunks into sentences
sentence_docs= {}
for doc in var_names:
    sentence_docs[doc]=sentence_split(chapter_chunks[doc])

In [15]:
#Cecking sentence from each docs
for doc in var_names:
    for i in range(len(sentence_docs[doc][:2])):
        print(f"\n{doc}_Sentence {i}:")
        print(sentence_docs[doc][i].get_content())
        #print("Title: {}".format(nodes[i].metadata.get("title")))
        #print("Window:")
        #print(nodes[i].metadata.get("window"))


MAS_Sentence 0:
1.1 The technology landscape of the financial sector is transforming at a rapid pace and the underlying information technology (IT) infrastructure supporting financial services has grown in scope and complexity in recent years.

MAS_Sentence 1:
Many financial institutions (FIs) are riding the wave of digitalisation to increase operational efficiency and to deliver better services to consumers.

RBI_Sentence 0:
1 Short Title and Commencement.

RBI_Sentence 1:
(a) These Directions shall be called the Reserve Bank of India (Information Technology Governance, Risk, Controls and Assurance Practices) Directions, 2023.

BB_Sentence 0:
The technology landscape of the financial sector is changing at a rapid pace and the underlying information technology (IT) infrastructure supporting financial services has grown in size and complexity in recent years.

BB_Sentence 1:
Digital transformation in the financial sector can be characterized by the introduction of new technologies and 

In [16]:
#Building index
index= {}
index_directory='../data/ict-policy/index'
for doc in var_names:
    if not os.path.exists(index_directory+f'/{doc}'):
        index[doc]=VectorStoreIndex.from_documents(sentence_docs[doc])
        index[doc].storage_context.persist(persist_dir=index_directory+f'/{doc}')
    else:
        index[doc]= load_index_from_storage(StorageContext.from_defaults(persist_dir=index_directory+f'/{doc}'))

In [17]:
from typing import List
from llama_index.core.indices.vector_store.retrievers import VectorIndexRetriever
from llama_index.core.schema import NodeWithScore, QueryBundle

class AllNodesUniqueTitleVectorIndexRetriever(VectorIndexRetriever):
    def __init__(self, *args, **kwargs):
        # Set similarity_top_k to a very large number to retrieve all nodes
        kwargs['similarity_top_k'] = 10000000000
        super().__init__(*args, **kwargs)
        self.chapter_chunks = kwargs.get('chapter_chunks', [])

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        # Call the parent class's _retrieve method to get all the initial results
        initial_results = super()._retrieve(query_bundle)

        # Filter out duplicate documents based on the "title" metadata
        seen_titles = set()
        unique_results = []
        for result in initial_results:
            title = result.node.metadata.get("title", None)
            if title and title not in seen_titles:
                seen_titles.add(title)
                unique_results.append(result)

        # Sort the unique results by score (if available) and select the top 5
        unique_results.sort(key=lambda x: x.score if x.score is not None else float('inf'), reverse=True)
        top_5_unique_results = unique_results[:5]

        # Replace text with window content after retrieval
        top_5_unique_results = self._replace_text_with_window_content(top_5_unique_results)

        return top_5_unique_results

    def _replace_text_with_window_content(self, results: List[NodeWithScore]) -> List[NodeWithScore]:
        for result in results:
            title = result.node.metadata.get("title")
            window_content = next((chunk["content"] for chunk in self.chapter_chunks if chunk["title"] == title), result.node.text)
            result.node.text = window_content
        return results


In [18]:
# Create an instance of the custom retriever
retrievers= {}
for doc in var_names:
    retrievers[doc] = AllNodesUniqueTitleVectorIndexRetriever(index[doc], chapter_chunks=chapter_chunks[doc])

In [19]:
#Creating query engine
from llama_index.core.query_engine.retriever_query_engine import RetrieverQueryEngine

query_engines= {}

for doc in var_names:
    query_engines[doc]= RetrieverQueryEngine.from_args(retrievers[doc])

In [20]:
query_engine_tool= [
    QueryEngineTool(
        query_engine=query_engines['RBI'],
        metadata= ToolMetadata(
            name="rbi_ICT_document",
            description=(
                "Provide ICT guidelines of Reserve Bank of India (RBI)."
                "Use a detailed plain text question as input to the tool."
            )
        )
    ),
    QueryEngineTool(
        query_engine=query_engines['MAS'],
        metadata= ToolMetadata(
            name="mas_ICT_document",
            description=(
                "Provide ICT guidelines of Monetary Authority of Singapore (MAS)."
                "Use a detailed plain text question as input to the tool."
            )
        )
    ),
    QueryEngineTool(
        query_engine=query_engines['BB'],
        metadata= ToolMetadata(
            name="bangladesh_bank_ICT_document",
            description=(
                "Provide ICT guidelines of Bangladesh Bank (BB)."
                "Use a detailed plain text question as input to the tool."
            )
        )
    ),    
]

In [17]:
from llama_index.core.agent import ReActAgent

agent= ReActAgent.from_tools(
    query_engine_tool, verbose=True, allow_parallel_tool_calls=True
)

In [18]:
response= agent.chat(
    "What provisions are made for penetration testing at MAS, RBI and BB. When giving final answer, summarize it with heading for each tool response."
)

> Running step 12039d78-bc2d-45cc-9819-75948f367d2e. Step input: What provisions are made for penetration testing at MAS, RBI and BB. When giving final answer, summarize it with heading for each tool response.
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step e8ed1b35-bca5-4a39-aacb-1e4c67a404fb. Step input: None
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: mas_ICT_document
Action Input: {'input': 'What provisions are made for penetration testing at MAS?'}
Observation: Penetration testing should be conducted to evaluate the effectiveness of cyber security defenses. This includes a combination of blackbox and greybox testing for online financial services. To ensure a thorough assessment, penetration testing should target the production environment with proper safeguards in place. The frequency of penetration testing should be determined based on the s

In [19]:
display(Markdown(response.response))

## Penetration Testing Provisions at MAS

Penetration testing should be conducted to evaluate the effectiveness of cyber security defenses. This includes a combination of blackbox and greybox testing for online financial services. To ensure a thorough assessment, penetration testing should target the production environment with proper safeguards in place. The frequency of penetration testing should be determined based on the system's criticality and exposure to cyber risks. For systems accessible from the Internet, penetration testing should be performed at least annually or after major changes. Additionally, a bug bounty program can be considered to complement penetration testing efforts.

## Penetration Testing Provisions at RBI

Penetration testing should be conducted at least annually for critical information systems and those in the De-Militarized Zone (DMZ) with customer interfaces. For non-critical systems, the frequency and necessity of penetration testing should be determined based on a risk-based approach. These tests should be performed by independent and trained information security experts or auditors. In post-implementation scenarios, penetration testing should ideally be done in the production environment, but if conducted in a test environment, it must closely resemble the production environment. Any deviations should be documented and approved by the Information Security Committee (ISC). Additionally, identified vulnerabilities should be addressed promptly to prevent recurrence.

## Penetration Testing Provisions at BB

Penetration testing at BB is addressed through a methodology that includes several key provisions:

- The methodology is based on industry-accepted penetration testing approaches, such as NIST SP800-115.
- It covers critical systems and includes testing from both inside and outside the network.
- It includes a review and consideration of threats and vulnerabilities experienced in the last 12 months.
- It specifies the retention of penetration testing results and remediation results.
- Penetration testing is performed at least annually and after any significant infrastructure or application upgrade or modification.

In [24]:
response= agent.chat(
    """What provisions are made for IT governance at MAS, RBI and BB. When giving final answer, summarize it with heading for each tool response."""
)

> Running step d2a8c98d-696c-4407-8ca9-62c97cd71db7. Step input: What provisions are made for IT governance at MAS, RBI and BB. When giving final answer, summarize it with heading for each tool response.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: mas_ICT_document
Action Input: {'input': 'What provisions are made for IT governance at MAS?'}
Observation: The provisions for IT governance include establishing a robust IT service management framework that supports IT services and operations, tracks information assets, manages changes, responds to incidents, and ensures the stability of the production IT environment. This framework comprises governance structure, processes, and procedures for various IT service management activities. Additionally, configuration management processes are implemented to maintain accurate information about hardware and software, and technology refresh management ensures that outdated hardwa

In [25]:
display(Markdown(response.response))

## IT Governance Provisions at MAS

The provisions for IT governance include establishing a robust IT service management framework that supports IT services and operations, tracks information assets, manages changes, responds to incidents, and ensures the stability of the production IT environment. This framework comprises governance structure, processes, and procedures for various IT service management activities. Additionally, configuration management processes are implemented to maintain accurate information about hardware and software, and technology refresh management ensures that outdated hardware and software are replaced to avoid security risks. Patch management, change management, software release management, incident management, and problem management processes are also established to maintain the security and stability of IT systems.

## IT Governance Provisions at RBI

The provisions for IT governance include establishing a robust IT Governance Framework that focuses on strategic alignment, risk management, resource management, performance management, and Business Continuity/Disaster Recovery Management. This framework specifies the governance structure and processes necessary to meet business objectives, defines roles and responsibilities of the Board of Directors and Senior Management, and includes oversight mechanisms to ensure accountability and mitigation of IT and cybersecurity risks. Additionally, enterprise-wide risk management policies must incorporate periodic assessment of IT-related risks.

## IT Governance Provisions at BB

Several provisions are made for IT governance. These include ensuring efficient and effective management of ICT functions and operations, with top management responsible for implementing appropriate IT security controls. A dedicated ICT Security Management oversees various aspects, including roles and responsibilities, ICT security policy, documentation, audits, training, awareness, insurance, and risk coverage.

Key stakeholders in ICT governance include the Board, ICT Steering Committee, ICT Security Committee, CEO/Managing Director, CIO, CTO, CITO, CISO, Risk Management Committee, Chief Risk Officer, and Business Executives. Each of these roles has specific responsibilities to ensure the alignment of ICT strategy with business objectives, monitor progress, provide guidance, and ensure compliance with regulatory requirements.

Additionally, the organization must have an approved ICT Security Policy that is reviewed annually and managed by dedicated ICT security professionals. Compliance with this policy is mandatory, and non-compliance issues must be addressed promptly, with potential penalties for failure to rectify within specified timeframes.

In [27]:
response= agent.chat(
    """What role of board is envisioned for IT governance at MAS, RBI and BB. When giving final answer, summarize it with heading for each tool response."""
)

> Running step 99c95331-da3c-4d8e-b5a1-7da37003bc2e. Step input: What role of board is envisioned for IT governance at MAS, RBI and BB. When giving final answer, summarize it with heading for each tool response.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: mas_ICT_document
Action Input: {'input': 'What role of board is envisioned for IT governance at MAS?'}
Observation: The board of directors is envisioned to play a pivotal role in IT governance at MAS. They are responsible for ensuring a sound and robust risk management framework is established and maintained to manage technology risks. This includes approving the risk appetite and risk tolerance statement, undertaking regular reviews of the technology risk management strategy, and assessing management competencies for managing technology risks. Additionally, the board should ensure an independent audit function is established to assess the effectiveness of control

In [28]:
display(Markdown(response.response))

## Role of Board in IT Governance at MAS

The board of directors is envisioned to play a pivotal role in IT governance at MAS. They are responsible for ensuring a sound and robust risk management framework is established and maintained to manage technology risks. This includes approving the risk appetite and risk tolerance statement, undertaking regular reviews of the technology risk management strategy, and assessing management competencies for managing technology risks. Additionally, the board should ensure an independent audit function is established to assess the effectiveness of controls, risk management, and governance of the FI. They are also accountable for giving senior executives sufficient authority, resources, and access to manage technology risks effectively.

## Role of Board in IT Governance at RBI

The Board of Directors is responsible for approving strategies and policies related to IT, Information Assets, Business Continuity, Information Security, and Cyber Security, including Incident Response and Recovery Management. These strategies and policies should be reviewed by the Board at least annually. Additionally, the Board oversees the establishment of a Board-level IT Strategy Committee (ITSC) to guide IT strategy and ensure effective IT governance and information security. The Board also reviews the adequacy and effectiveness of Business Continuity Planning and Disaster Recovery Management on an annual basis.

## Role of Board in IT Governance at BB

The Board has several key roles in IT governance:

- **Approving ICT Strategy and Policies**: The Board is responsible for approving the ICT strategy and policies to ensure they align with the business objectives and strategy.
- **Ensuring Effective Planning**: The Board must ensure that management has an effective planning process in place.
- **Aligning ICT Strategy with Business Strategy**: The Board endorses the alignment of the ICT strategy with the business strategy.
- **Overseeing ICT Organizational Structure**: The Board ensures that the ICT organizational structure complements the business model and its direction.
- **Balancing ICT Investments**: The Board ensures that ICT investments represent a balance of risks and benefits with acceptable budgets.
- **Ensuring Accountability**: The Board ensures accountability within the ICT governance framework.
- **Monitoring Compliance**: